
# 한국 투자 증권 API 테스트

## idxcode.mst -> idxcode.csv

`idxcode.mst`는 KIS에서 제공하는 **업종 코드 목록** 데이터입니다:    
즉시 다운로드 링크: https://new.real.download.dws.co.kr/common/master/idxcode.mst.zip  
mst -> csv 로 변환 및 즉시 사용가능한 형태로 바꾸려면 `1_kis_index_to_excel.ipynb`를 실행하세요.  
  
**각종 종목 파일 종합 링크:**  
https://apiportal.koreainvestment.com/apiservice-category  


In [ ]:
## 초기화 블럭

import os
import sys
from pathlib import Path
from pprint import pprint
sys.path.append('../../')
from packages.kis_api.src.kis_api import (
    KISClient, 
    fetch_inquire_index_price,
    fetch_inquire_price
)

from datetime import datetime
from typing import Any, List, Mapping, Optional

# DOTENV = '../packages/kis_api/.env'

def load_env_file(path: Path | None = None) -> None:
    """간단한 .env 로더 (tests/conftest.py와 동일한 규칙)."""
    target = path or Path.cwd() / ".env"
    if not target.is_file():
        return
    for raw_line in target.read_text().splitlines():
        line = raw_line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        key, value = line.split("=", 1)
        normalized = value.strip().strip('\"').strip("'")
        os.environ.setdefault(key.strip(), normalized)

load_env_file()

# KISClient 인스턴스 생성
app_key = os.getenv("KIS_APP_KEY")
app_secret = os.getenv("KIS_APP_SECRET")
if not app_key or not app_secret:
    raise RuntimeError(".env에 KIS_APP_KEY/KIS_APP_SECRET를 설정해주세요")

base_url = os.getenv("KIS_BASE_URL", "https://openapi.koreainvestment.com:9443")
timeout = float(os.getenv("KIS_TIMEOUT", "10"))

CLIENT = KISClient(
    app_key=app_key, app_secret=app_secret, base_url=base_url, timeout=timeout
)

In [ ]:


fetch_inquire_index_price(CLIENT, "0195")
# fetch_inquire_price(CLIENT, "005930")



In [ ]:
# 주식 정보 Fetch
# https://apiportal.koreainvestment.com/testbed-intro?group=%5B%EA%B5%AD%EB%82%B4%EC%A3%BC%EC%8B%9D%5D%20%EC%A2%85%EB%AA%A9%EC%A0%95%EB%B3%B4&name=%EC%A3%BC%EC%8B%9D%EA%B8%B0%EB%B3%B8%EC%A1%B0%ED%9A%8C%5Bv1_%EA%B5%AD%EB%82%B4%EC%A3%BC%EC%8B%9D-067%5D
kis_stock_korea_info_endpoint = "/uapi/domestic-stock/v1/quotations/search-stock-info"
params = {
    "PRDT_TYPE_CD": "300",
    "PDNO": "000660",
}

result = CLIENT.request('GET', 
               kis_stock_korea_info_endpoint, 
               params=params,
               headers={"tr_id": "CTPF1002R", "custtype": "P"})
print(result)
